# Intelligente Agenten
## Der Turing-Test
1. Wie funktioniert der Turing-Test?
2. Was braucht eine Maschine um den Test zu bestehen?
3. Was ist der totale Turing-Test?
4. Was braucht eine Maschine um den totalen Turing-Test zu bestehen?
5. Welche Probleme existieren beim Turing-Test?

## Agenten
1. Was ist ein Agent? Aus welchen Bestandteilen besteht ein Agent?
2. Diskutieren Sie die folgenden Begriffe. Geben Sie jeweils 2 Beispiele für:<br/>
    a) Vollständig beobachtbare Umgebung.<br/>
    b) Nichtdeterministische Umgebung.<br/>
    c) Single-Agent Umgebung.<br/>
    d) Multi-Agent Umgebung.<br/>
    e) Umgebung mit diskretem Zustandsraum.<br/>
    f) Umgebung mit kontinuierlichem Zustandsraum.<br/>
    g) Einfache Reflex-Agenten.<br/>
    h) Modellbasierte Reflex-Agenten.<br/>
    i) Zielbasierte Agenten.<br/>
    j) Lernfähige Agenten.<br/>
3. Was ist Rationalität?

## Der Staubsauger als Agent
Wir werden heute einen Staubsauger-Agenten und die dazugehörige Umgebung bzw. "Welt" programmieren.

In [ ]:
import random

# (x, y) coordinates of the different locations 
loc_A, loc_B = (0, 0), (1, 0) 

Zunächst definieren wir uns eine Klasse "Thing". Diese Klasse repräsentiert alle physischen Objekte, die in unserer Welt vorkommen können. Dabei kann ein Gegenstand entweder lebendig sein oder nicht.

In [ ]:
class Thing:
    """This represents any physical object that can appear in an Environment.
    You subclass Thing to get the things you want. Each thing can have a
    .__name__  slot (used for output only)."""

    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))

    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Display the agent's internal state. Subclasses should override."""
        print("I don't know how to show_state.")

    def display(self, canvas, x, y, width, height):
        """Display an image of this Thing on the canvas."""
        # Do we need this?
        pass

Die Thing-Klasse benötigen wir dann später auch für unsere Umgebung. Zu diesem Zweck wird zunächst eine abstrakte Klasse "Environment" definiert, welche als "Dummy" für spätere konkrete Implementierungen verschiedener Umgebungen bzw. Welten dient.

In [ ]:
class Environment:
    """Abstract class representing an Environment. 'Real' Environment classes
    inherit from this. Your Environment will typically need to implement:
        percept:           Define the percept that an agent sees.
        execute_action:    Define the effects of executing an action.
                           Also update the agent.performance slot.
    The environment keeps a list of .things and .agents (which is a subset
    of .things). Each agent has a .performance slot, initialized to 0.
    Each thing has a .location slot, even though some environments may not
    need this."""

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Return the percept that the agent sees at this point. (Implement this.)"""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """Change the world to reflect this action. (Implement this.)"""
        raise NotImplementedError

    def default_location(self, thing):
        """Default location to place a new thing with unspecified location."""
        return None

    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        actions = []
        for agent in self.agents:
            actions.append(agent.program(self.percept(agent)))
        for (agent, action) in zip(self.agents, actions):
            self.execute_action(agent, action)
        self.exogenous_change()

    def run(self, steps=1000):
        """Run the Environment for given number of time steps."""
        for step in range(steps):
            self.step()

    def list_things_at(self, location, tclass=Thing):
        """Return all things exactly at a given location."""
        return [thing for thing in self.things
                if thing.location == location and isinstance(thing, tclass)]

    def some_things_at(self, location, tclass=Thing):
        """Return true if at least one of the things at location
        is an instance of class tclass (or a subclass)."""
        return self.list_things_at(location, tclass) != []

    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)

    def delete_thing(self, thing):
        """Remove a thing from the environment."""
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print("  in Environment delete_thing")
            print("  Thing to be removed: {} at {}".format(thing, thing.location))
            print("  from list: {}".format([(thing, thing.location) for thing in self.things]))
        if thing in self.agents:
            self.agents.remove(thing)

Nun da wir uns abstrakten Klassen für Objekte und mögliche Welten programmiert haben, können wir damit beginnen eine konkrete Welt zu implementieren. Bei der folgenden Klasse handelt es sich um eine triviale Welt für einen Staubsauger-Roboter. Sie besteht aus nur zwei Locations (loc_A, loc_B) die beide jeweils nebeneinander liegen. Jede Location kann dabei entweder dreckig oder sauber sein.

Weiterhin können in unserer Welt Wände, Schmutz und verschiedene Agenten als "Things" enthalten sein (thing_classes-Methode). Ebenfalls wird definiert, was der Agent in unserer Welt warnehmen kann (percept-Methode) und wie sich die möglichen Aktionen unseres Agenten auf die Welt auswirken (execute_action-Methode).

In [ ]:
class TrivialVacuumEnvironment(Environment):

    """This environment has two locations, A and B. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status. This serves as an example of how to implement a simple
    Environment."""

    def __init__(self):
        super().__init__()
        self.status = {loc_A: random.choice(['Clean', 'Dirty']),
                       loc_B: random.choice(['Clean', 'Dirty'])}

    def thing_classes(self):
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        return (agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status; track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        if action == 'Right':
            agent.location = loc_B
            agent.performance -= 1
        elif action == 'Left':
            agent.location = loc_A
            agent.performance -= 1
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                agent.performance += 10
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B])

Nachdem wir uns nun mit der Implementierung der "Welt" und ihrer möglichen Ausprägungen auseinandergesetzt haben, kommen wir nun zu den eigentlichen Agenten. Dazu schaffen wir uns zunächst eine Klasse "Agent", welche wiederum eine Unterklasse von "Thing" ist. 

Dabei hat unser Agent nur ein einzelnes Attribut, welches das eigentlichen "Verhalten" des Agenten enthält. Beim Initialisieren eines neuen Agenten, muss diese "Verhalten" in Form einer Python-Funktion an die Agenten-Klasse übergeben werden. Dabei ist darauf zu achten, dass die übergebene Funktion als Argument eine "Wahrnehmung" bekommt und als Rückgabe eine (in der jeweiligen Welt mögliche) Aktion zurückliefert.

In [ ]:
class Agent(Thing):
    """An Agent is a subclass of Thing with one required slot,
    .program, which should hold a function that takes one argument, the
    percept, and returns an action. (What counts as a percept or action
    will depend on the specific environment in which the agent exists.)
    Note that 'program' is a slot, not a method. If it were a method,
    then the program could 'cheat' and look at aspects of the agent.
    It's not supposed to do that: the program can only look at the
    percepts. An agent program that needs a model of the world (and of
    the agent itself) will have to build and maintain its own model.
    There is an optional slot, .performance, which is a number giving
    the performance measure of the agent in its environment."""
    def __init__(self, program):
        self.program = program

Diese Funktion ermöglicht lediglich die Ausgabe der jeweiligen Wahrnehmung des Agenten und der daraus resultierenden Aktion.

In [ ]:
def TraceAgent(agent):
    """Wrap the agent's program to print its input and output. This will let
    you see what the agent is doing in the environment."""
    old_program = agent.program

    def new_program(percept):
        action = old_program(percept)
        print('{} perceives {} and does {}'.format(agent, percept, action))
        return action
    agent.program = new_program
    return agent

Nun definieren wir uns einen einfachen Agenten, der nichts macht. 

In [ ]:
def SimpleAgent():
    """A simple agent for the two-state vacuum environment which does nothing."""
    def program(percept):
        loc, status = percept
        return None
    return Agent(program)

Hier nun ein Beispiel wie wir unseren Agenten aufrufen können:

In [ ]:
a = TraceAgent(SimpleAgent())
a.program((loc_A, 'Clean'))
a.program((loc_B, 'Clean'))
a.program((loc_A, 'Dirty'))
a.program((loc_A, 'Dirty'))

Dabei haben wir allerdings noch keine Welt vorgegeben, sondern dem Agenten lediglich mit jedem Aufruf jeweils einen möglichen Zustand unserer zweidimensionalen Welt vorgegeben.

Um unseren Agenten in der von uns definierten Umgebung interagieren zu lassen, müssen wir nun folgendes tun:

In [ ]:
# initialize environment
e = TrivialVacuumEnvironment()
# add an agent to our environment
e.add_thing(TraceAgent(SimpleAgent()))
# execute our environment and the agent for five discrete timesteps
e.run(5)

## Aufgabe 1
Implementieren Sie die Klasse ReflexVacuumAgent als einfachen Reflex-Agenten. Das Verhalten können sie dabei selbst festlegen. Der Agent soll dabei in unserer Welt "TrivialVacuumEnvironment" funktionieren. Beachten Sie diesen Umstand bei der Festlegung möglicher Aktionen, die der Agent in der genannten Welt ausführen soll.

In [ ]:
def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment."""
    def program(percept):
        loc, status = percept
        # TODO Aufgabe 1
        return None
    return Agent(program)

In [ ]:
# initialize environment
e = TrivialVacuumEnvironment()
# add an agent to our environment
e.add_thing(TraceAgent(ReflexVacuumAgent()))
# execute our environment and the agent for five discrete timesteps
e.run(5)

## Aufgabe 2
Implementieren Sie die Klasse "ModelBasedVacuumAgent" als modellbasierten Agenten. Auch dieser Agent soll in der "TrivialVacuumEnvironment" funktionieren. 

In [ ]:
def ModelBasedVacuumAgent():
    """An agent that keeps track of what locations are clean or dirty."""
    model = {loc_A: None, loc_B: None}

    def program(percept):
        location, status = percept
        # TODO Aufgabe 2
        return None
    return Agent(program)

In [ ]:
# initialize environment
e = TrivialVacuumEnvironment()
# add an agent to our environment
e.add_thing(TraceAgent(ModelBasedVacuumAgent()))
# execute our environment and the agent for five discrete timesteps
e.run(5)

## Aufgabe 3
Erweitern Sie die Implementierung der VacuumEnvironment, sodass es beliebig viele, auf einem 2D-Gitter angeordnete Positionen gibt. Erweitern Sie auch die Implementierung der beiden Agenten entsprechend, sodass diese sinnvoll in der neuen Umgebung agieren können

In [ ]:
class ExtendedTrivialVacuumEnvironment(Environment):

    """This environment has two locations, A and B. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status. This serves as an example of how to implement a simple
    Environment."""

    def __init__(self):
        super().__init__()
        # Todo
        self.status = None

    def thing_classes(self):
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        return (agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status; track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        #Todo
        pass

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B])